# Clients Cleaning

## Load Libraries

In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from datetime import datetime

## Load Dataset

In [2]:
# Paths to datasets
clients_vm1_path = '../data/clients-vm1.csv'
clients_vm2_path = '../data/clients-vm2.csv'

df_clients = pd.concat(map(pd.read_csv, [clients_vm1_path, clients_vm2_path]))
df_clients.head()

,cursor,owner,name,sshUrl,url,stars,createdAt,pushedAt,packages,groupId,artifactId,version,path,clients,relevantClients,cowner,cname,csshUrl,curl,cstars
0,Y3Vyc29yOjY=,javaparser,javaparser,ssh://git@github.com:javaparser/javaparser.git,https://github.com/javaparser/javaparser,4290,2011-10-30,2022-07-27,5,com.github.javaparser,javaparser-core,3.24.3-SNAPSHOT,javaparser-core/,3166,27,kiegroup,optaplanner,ssh://git@github.com:kiegroup/optaplanner.git,https://github.com/kiegroup/optaplanner,2676
1,Y3Vyc29yOjY=,javaparser,javaparser,ssh://git@github.com:javaparser/javaparser.git,https://github.com/javaparser/javaparser,4290,2011-10-30,2022-07-27,5,com.github.javaparser,javaparser-core,3.24.3-SNAPSHOT,javaparser-core/,3166,27,Azure,azure-sdk-for-java,ssh://git@github.com:Azure/azure-sdk-for-java.git,https://github.com/Azure/azure-sdk-for-java,1556
2,Y3Vyc29yOjY=,javaparser,javaparser,ssh://git@github.com:javaparser/javaparser.git,https://github.com/javaparser/javaparser,4290,2011-10-30,2022-07-27,5,com.github.javaparser,javaparser-core,3.24.3-SNAPSHOT,javaparser-core/,3166,27,fabric8io,kubernetes-client,ssh://git@github.com:fabric8io/kubernetes-clie...,https://github.com/fabric8io/kubernetes-client,2579
3,Y3Vyc29yOjY=,javaparser,javaparser,ssh://git@github.com:javaparser/javaparser.git,https://github.com/javaparser/javaparser,4290,2011-10-30,2022-07-27,5,com.github.javaparser,javaparser-core,3.24.3-SNAPSHOT,javaparser-core/,3166,27,abstracta,jmeter-java-dsl,ssh://git@github.com:abstracta/jmeter-java-dsl...,https://github.com/abstracta/jmeter-java-dsl,186
4,Y3Vyc29yOjY=,javaparser,javaparser,ssh://git@github.com:javaparser/javaparser.git,https://github.com/javaparser/javaparser,4290,2011-10-30,2022-07-27,5,com.github.javaparser,javaparser-core,3.24.3-SNAPSHOT,javaparser-core/,3166,27,kiegroup,drools,ssh://git@github.com:kiegroup/drools.git,https://github.com/kiegroup/drools,4563


## Validate Data

In [3]:
rows, columns = df_clients.shape
df_clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3238 entries, 0 to 1627
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cursor           3238 non-null   object
 1   owner            3238 non-null   object
 2   name             3238 non-null   object
 3   sshUrl           3238 non-null   object
 4   url              3238 non-null   object
 5   stars            3238 non-null   int64 
 6   createdAt        3238 non-null   object
 7   pushedAt         3238 non-null   object
 8   packages         3238 non-null   int64 
 9   groupId          3238 non-null   object
 10  artifactId       3238 non-null   object
 11  version          3238 non-null   object
 12  path             2900 non-null   object
 13  clients          3238 non-null   int64 
 14  relevantClients  3238 non-null   int64 
 15  cowner           3238 non-null   object
 16  cname            3238 non-null   object
 17  csshUrl          3238 non-null   

All columns seem to have non-null values except for the `path` one. We need to further investigate these cases.


### Sanity Checks

**Auxiliary Functions**

In [4]:
def compute_url_parts(url_col: str) -> tuple[str, str]:
    """
    Returns the GitHub scheme and authority of the clients data 
    frame column.
    :param url_col: name of the column in the data frame. Can 
        be 'sshUrl', 'url', 'csshUrl', or 'curl'
    :returns: tuple whose first element is the scheme of the
        expected urls (e.g. 'ssh', 'https') and its second 
        element is the authority of the expected urls.
    :raises: `ValueError` if the url_col is invalid.
    """
    if url_col in ['sshUrl', 'csshUrl']:
        return ('ssh', 'git@github.com')
    elif url_col in ['url', 'curl']:
        return ('https', 'github.com')
    else:
        raise Exception('Invalid url_col value')
        
        
def validate_urls(url_col: str, df: pd.DataFrame) -> bool:
    """
    Validates if the values of a given URL column within the clients
    data frame are valid. Concretely, it performs three checks:
    1. all URLs start with the expected scheme; 2. all URLs 
    contain the expected authority, and; 3. all URLs are built 
    in the right way and contain the expected path based on the 
    repository owner and name values.
    :param url_col: name of the column in the data frame. Can 
        be 'sshUrl', 'url', 'csshUrl', or 'curl'
    :param df: clients data frame
    :returns: `True` if all checks pass, `False` otherwise.
    """
    # Define owner and name column names
    owner_col = 'owner'
    name_col = 'name'
    if url_col.startswith('c'):
        owner_col = 'c' + owner_col
        name_col = 'c' + name_col
        
    # E.g., ssh://git@github.com:owner/name
    # or https://github.com/owner/name
    scheme, authority = compute_url_parts(url_col)
    delimiter = ':' if scheme == 'ssh' else '/'
    extension = '.git' if scheme == 'ssh' else ''
    
    # SSH URLs have the right scheme
    mask = df[url_col].str.match(f'^{scheme}://')
    scheme_check = mask.all()
    print(f'Records have right scheme: {scheme_check}')

    # SSH URLs have the right authority
    mask = df[url_col].str.contains(f'//{authority}{delimiter}')
    authority_check = mask.all()
    print(f'Records have right authority: {authority_check}')

    # SSH URLs have the right path
    mask = df[url_col] == f'{scheme}://{authority}{delimiter}' + df[owner_col].map(str) + '/' + df[name_col].map(str) + extension
    path_check = mask.all()
    print(f'Records have right path: {path_check}')
    
    return scheme_check and authority_check and path_check


def validate_date(date_col: str, df: pd.DataFrame) -> bool:
    """
    Validates if the values of a given date column within the 
    clients data frame are valid. Concretely, it checks if all 
    dates are of the format yyyy-MM-dd.
    :param date_col: name of the column in the data frame. Can 
        be 'createdAt' or 'pushedAt'
    :param df: clients data frame
    :returns: `True` if the check passes, `False` otherwise.
    :raises: `ValueError` if the date_col is invalid.
    """
    if date_col not in ['createdAt', 'pushedAt']:
        raise ValueError('Invalid date_col value')
        
    mask = df[date_col].str.match('^\d{4}-\d{2}-\d{2}')
    format_check = mask.all()
    print(f'Records have right format: {format_check}')
    
    return format_check


def validate_limit_date(limit_date: datetime, date_col: str, df: pd.DataFrame) -> bool:
    """
    Validates if the values of a given date column within the 
    clients data frame are valid. Concretely, it performs two 
    checks: 1. all date appear after a limit date, and; 2. all 
    dates appear before the present date.
    :param limit_date: limit past date for all records
    :param date_col: name of the column in the data frame. Can 
        be 'createdAt' or 'pushedAt'
    :param df: clients data frame
    :returns: `True` if all checks pass, `False` otherwise.
    :raises: `ValueError` if the date_col is invalid.
    """ 
    if date_col not in ['createdAt', 'pushedAt']:
        raise ValueError('Invalid date_col value')
    
    mask = pd.to_datetime(df[date_col]) > limit_date
    limit_check = mask.all()
    print(f'Records appear after {limit_date}: {limit_check}')
    
    present = datetime.now()
    mask = pd.to_datetime(df[date_col]) <= present
    present_check = mask.all()
    print(f'Records appear before {present}: {present_check}')
    
    return limit_check and present_check


def validate_limit_num(limit_num: int, num_col: str, df: pd.DataFrame) -> bool:
    """
    """
    mask = df[num_col] >= limit_num
    limit_check = mask.all()
    print(f'Records have at least {limit_num}: {limit_check}')
    
    return limit_check

**Validate `sshUrl` column:** Verify that all SSH URLs are correctly build and match the expected repository owner/name.

In [5]:
ssh_url_check = validate_urls('sshUrl', df_clients)
assert ssh_url_check, 'sshUrl check'

Records have right scheme: True
Records have right authority: True
Records have right path: True


**Validate `url` column:** Verify that all HTTP URLs are correctly build and match the expected repository owner/name.

In [6]:
http_url_check = validate_urls('url', df_clients)
assert http_url_check, 'url check'

Records have right scheme: True
Records have right authority: True
Records have right path: True


**Validate `stars` column:** Verify that all records have at least 2,500 stars.

In [7]:
min_stars = 2500
stars_check = validate_limit_num(min_stars, 'stars', df_clients)
assert stars_check, 'stars check'

Records have at least 2500: True


**Validate `createdAt` column:** Verify that all dates have the expected format "yyyy-MM-dd".

In [8]:
last_pushed_date = datetime(2022, 7, 22)
created_at_check = validate_date('createdAt', df_clients)
assert created_at_check, 'createdAt check'

Records have right format: True


**Validate `pushedAt` column:** Verify that all dates have the expected format "yyyy-MM-dd" and appear after a limit date.

In [9]:
last_pushed_date = datetime(2022, 4, 22)
pushed_at_format_check = validate_date('pushedAt', df_clients)
pushed_at_limit_check = validate_limit_date(last_pushed_date, 'pushedAt', df_clients)
assert pushed_at_format_check and pushed_at_limit_check, 'pushedAt check'

Records have right format: True
Records appear after 2022-04-22 00:00:00: True
Records appear before 2022-08-07 19:10:39.275193: True


**Validate `packages` column:** Verify that all records have more than 0 packages.

In [10]:
packages_check = validate_limit_num(1, 'packages', df_clients)
assert packages_check, 'packages check'

Records have at least 1: True


**Validate `path` column:** The `info` method already gave some insights about records where the package path in `null`. We will further investigate these cases, and check if it is necessary to exclude them from the dataset.

In [11]:
mask = df_clients['path'].isnull()
print(f'Records in the "path" column with null values: {mask.sum()}')

Records in the "path" column with null values: 338


In [12]:
# Unique combinations
df_empty_path = df_clients[mask]
df_empty_path.groupby(['url', 'groupId', 'artifactId', 'version']).size()

url                                                    groupId                    artifactId            version               
https://github.com/JodaOrg/joda-time                   joda-time                  joda-time             2.10.14                    7
https://github.com/LWJGL/lwjgl3                        org.lwjgl                  lwjgl                 3.2.3                      8
https://github.com/OpenHFT/Chronicle-Queue             net.openhft                chronicle-queue       5.23ea30-SNAPSHOT          4
https://github.com/alibaba/QLExpress                   com.alibaba                QLExpress             3.3.0                      3
https://github.com/alibaba/fastjson                    com.alibaba                fastjson              1.2.84-SNAPSHOT           37
https://github.com/apache/curator                      org.apache.curator         apache-curator        5.4.0-SNAPSHOT             3
https://github.com/apache/hbase                        org.apache.hbase    

**Analysis of `path` null cases:** All POM files exist in the corresponding repository and they are placed at the root of the folder. A check of the datasets code is required to identify the source of this issue.

| Repository | POM exists? | Path | 
|------------|:-----------:|------|
| https://github.com/JodaOrg/joda-time | True | / |
| https://github.com/LWJGL/lwjgl3 | True | / |
| https://github.com/OpenHFT/Chronicle-Queue | True | / |
| https://github.com/alibaba/QLExpress | True | / |
| https://github.com/alibaba/fastjson | True | / |
| https://github.com/apache/curator | True | / |
| https://github.com/apache/hbase | True | / |
| https://github.com/apache/logging-log4j2  | True | / |
| https://github.com/apache/nifi | True | / |
| https://github.com/apache/rocketmq | True | / |
| https://github.com/apache/storm | True | / |
| https://github.com/apache/zookeeper | True | / |
| https://github.com/brettwooldridge/HikariCP | True | / |
| https://github.com/bytedeco/javacpp  | True | / | 
| https://github.com/bytedeco/javacv | True | / |
| https://github.com/checkstyle/checkstyle | True | / |
| https://github.com/eclipse-vertx/vert.x | True | / |
| https://github.com/google/flatbuffers | True | / |
| https://github.com/hs-web/hsweb-framework | True | / |
| https://github.com/jhy/jsoup | True | / |
| https://github.com/jvm-profiling-tools/async-profiler | True | / |
| https://github.com/mybatis/mybatis-3 | True | / |
| https://github.com/pmd/pmd | True | / |
| https://github.com/redis/jedis | True | / |
| https://github.com/spring-cloud/spring-cloud-netflix | True | / |
| https://github.com/square/javapoet | True | / |
| https://github.com/stanfordnlp/CoreNLP | True | / |

As an additional remark, no package registers paths pointing to the root of the repository. This cases must be considered in the code.

In [13]:
mask = df_clients['path'] == '/'
print(f'Number of packages at the root of the repository: {df_clients[mask].shape[0]}')

Number of packages at the root of the repository: 0


**Validate `clients` column:** Verify that all records have more than 0 clients and that the number of total clients is greater than the number of relevant clients.

In [14]:
clients_check = validate_limit_num(1, 'clients', df_clients)

mask = df_clients['clients'] >= df_clients['relevantClients']
limit_check = mask.all()
print(f'Records have more clients than relevant clients: {limit_check}')

assert clients_check and limit_check, 'clients check'

Records have at least 1: True
Records have more clients than relevant clients: True


**Validate `relevantClients` column:** Verify that all records have more than 0 relevant clients.

In [15]:
relevant_clients_check = validate_limit_num(1, 'relevantClients', df_clients)
assert relevant_clients_check, 'relevantClients check'

Records have at least 1: True


**Validate `csshUrl` column:** Verify that all client SSH URLs are correctly build and match the expected repository owner/name.

In [16]:
cssh_url_check = validate_urls('csshUrl', df_clients)
assert cssh_url_check, 'csshUrl check'

Records have right scheme: True
Records have right authority: True
Records have right path: True


**Validate `curl` column:** Verify that all client HTTP URLs are correctly build and match the expected repository owner/name.

In [17]:
chttp_url_check = validate_urls('curl', df_clients)
assert chttp_url_check, 'curl check'

Records have right scheme: True
Records have right authority: True
Records have right path: True


**Validate `cstars` column:** Verify that all client records have at least 100 stars.

In [18]:
min_stars = 100
cstars_check = validate_limit_num(min_stars, 'cstars', df_clients)
assert cstars_check, 'cstars check'

Records have at least 100: True


## Clean Dataset

**Remove pushed before 2022-05-01 cases:** Only keep records where a pushed to the library repository is registered from 2022-05-01.

In [19]:
last_pushed_date = datetime(2022, 5, 1)
initial_rows = df_clients.shape[0]

mask = pd.to_datetime(df_clients['pushedAt']) >= last_pushed_date
df_clients = df_clients[mask]
final_rows = df_clients.shape[0]

print(f'Pushed after 2022-05-01: {initial_rows - final_rows} have been removed')

Pushed after 2022-05-01: 6 have been removed


**Replace null `path` values by `/`:** Replaces all null values by the root of the directory after performing a manual review.

In [20]:
path_null_mask = df_clients['path'].isnull()
modified_rows = df_clients[path_null_mask].shape[0]

df_clients['path'].fillna('/', inplace=True)
mask = ~df_clients['path'].isnull()
path_null_check = mask.all()
assert path_null_check, 'path null check'

print(f'Replace null path values by /: {modified_rows} have been updated')
df_clients[path_null_mask].head()

Replace null path values by /: 338 have been updated


,cursor,owner,name,sshUrl,url,stars,createdAt,pushedAt,packages,groupId,artifactId,version,path,clients,relevantClients,cowner,cname,csshUrl,curl,cstars
88,Y3Vyc29yOjIx,bytedeco,javacpp,ssh://git@github.com:bytedeco/javacpp.git,https://github.com/bytedeco/javacpp,3893,2014-04-21,2022-07-08,3,org.bytedeco,javacpp,1.5.8-SNAPSHOT,/,1057,5,itboyst,ArcSoftFaceDemo,ssh://git@github.com:itboyst/ArcSoftFaceDemo.git,https://github.com/itboyst/ArcSoftFaceDemo,275
89,Y3Vyc29yOjIx,bytedeco,javacpp,ssh://git@github.com:bytedeco/javacpp.git,https://github.com/bytedeco/javacpp,3893,2014-04-21,2022-07-08,3,org.bytedeco,javacpp,1.5.8-SNAPSHOT,/,1057,5,SonicCloudOrg,sonic-server,ssh://git@github.com:SonicCloudOrg/sonic-serve...,https://github.com/SonicCloudOrg/sonic-server,1311
90,Y3Vyc29yOjIx,bytedeco,javacpp,ssh://git@github.com:bytedeco/javacpp.git,https://github.com/bytedeco/javacpp,3893,2014-04-21,2022-07-08,3,org.bytedeco,javacpp,1.5.8-SNAPSHOT,/,1057,5,SonicCloudOrg,sonic-agent,ssh://git@github.com:SonicCloudOrg/sonic-agent...,https://github.com/SonicCloudOrg/sonic-agent,952
91,Y3Vyc29yOjIx,bytedeco,javacpp,ssh://git@github.com:bytedeco/javacpp.git,https://github.com/bytedeco/javacpp,3893,2014-04-21,2022-07-08,3,org.bytedeco,javacpp,1.5.8-SNAPSHOT,/,1057,5,meteoinfo,MeteoInfo,ssh://git@github.com:meteoinfo/MeteoInfo.git,https://github.com/meteoinfo/MeteoInfo,217
92,Y3Vyc29yOjIx,bytedeco,javacpp,ssh://git@github.com:bytedeco/javacpp.git,https://github.com/bytedeco/javacpp,3893,2014-04-21,2022-07-08,3,org.bytedeco,javacpp,1.5.8-SNAPSHOT,/,1057,5,gliwka,hyperscan-java,ssh://git@github.com:gliwka/hyperscan-java.git,https://github.com/gliwka/hyperscan-java,116


## Store Cleaned Dataset

In [21]:
output_path = '../data/clients-cleaned.csv'
df_clients.to_csv(output_path)